# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('consult').getOrCreate()

In [2]:
data = spark.read.csv('hack_data.csv',header=True,inferSchema=True)

In [3]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [4]:
data.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [5]:
from pyspark.ml.feature import VectorAssembler,StringIndexer

In [11]:
##data.groupBy('Location')

In [12]:
indexer = StringIndexer(inputCol='Location',outputCol='IndexLoc')

In [13]:
indexData = indexer.fit(data)

In [15]:
index_Data = indexData.transform(data)

In [16]:
index_Data.select('IndexLoc').show()

+--------+
|IndexLoc|
+--------+
|    54.0|
|    37.0|
|    63.0|
|   161.0|
|    15.0|
|   169.0|
|    52.0|
|    40.0|
|    13.0|
|    94.0|
|    21.0|
|    29.0|
|    31.0|
|    78.0|
|    19.0|
|   170.0|
|     2.0|
|    36.0|
|    61.0|
|   120.0|
+--------+
only showing top 20 rows



In [17]:
index_Data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'IndexLoc']

In [18]:
from pyspark.ml.clustering import KMeans

In [20]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed', 'IndexLoc'],outputCol='features')

In [21]:
assembled_data = assembler.transform(index_Data)

In [23]:
assembled_data.select('features').show()

+--------------------+
|            features|
+--------------------+
|[8.0,391.09,1.0,2...|
|[20.0,720.99,0.0,...|
|[31.0,356.32,1.0,...|
|[2.0,228.08,1.0,2...|
|[20.0,408.5,0.0,3...|
|[1.0,390.69,1.0,2...|
|[18.0,342.97,1.0,...|
|[22.0,101.61,1.0,...|
|[15.0,275.53,1.0,...|
|[12.0,424.83,1.0,...|
|[15.0,249.09,1.0,...|
|[32.0,242.48,0.0,...|
|[23.0,514.54,0.0,...|
|[9.0,284.77,0.0,3...|
|[27.0,779.25,1.0,...|
|[12.0,307.31,1.0,...|
|[21.0,355.94,1.0,...|
|[10.0,372.65,0.0,...|
|[20.0,347.23,1.0,...|
|[22.0,456.57,0.0,...|
+--------------------+
only showing top 20 rows



In [24]:
kmean = KMeans(k=2,featuresCol='features')

In [25]:
k_model = kmean.fit(assembled_data)

In [26]:
new_data = k_model.transform(assembled_data)

In [31]:
new_data1 = new_data.select('prediction')

In [38]:
new_data.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  171|
|         0|  163|
+----------+-----+



In [43]:
kmean3 = KMeans(k=3,featuresCol='features')

In [44]:
k_model3 = kmean3.fit(assembled_data)

In [45]:
new_data3 = k_model3.transform(assembled_data)

In [46]:
new_data3.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   99|
|         2|   91|
|         0|  144|
+----------+-----+



## 2 hackers after verifying the clusters